In [39]:
import os

from dotenv import load_dotenv

load_dotenv()

True

## Agent

In [49]:
import json
from datetime import datetime

import requests
from langchain.agents import initialize_agent, Tool

# Define tools as functions
def read_tasks_from_file(input):
    with open('tasks.json', 'r') as file:
        tasks = json.load(file)
    return tasks

def check_weather(date):
    response = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/London,UK/{datetime.now().strftime('%Y-%m-%d')}?key={os.getenv('WEATHER_API_KEY')}")
    return response.json()

def add_to_calendar(task):
    print(f"Adding task to calendar: {task}")
    return f"Task {task} added to calendar."

# Define the tools for the agent
tools = [
    Tool(
        name="read_tasks_from_file",
        func=read_tasks_from_file,
        description="Reads tasks from a local JSON file"
    ),
    Tool(
        name="check_weather",
        func=check_weather,
        description="Checks the weather forecast for a given date"
    ),
    Tool(
        name="add_to_calendar",
        func=add_to_calendar,
        description="Adds a task to the calendar"
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


sample_tasks = [
    {"title": "Buy groceries", "tags": ["shopping"]},
    {"title": "Jogging", "tags": ["outdoor", "exercise"]},
    {"title": "Team meeting", "tags": ["work"]},
]
with open('tasks.json', 'w') as file:
    json.dump(sample_tasks, file)

input_text = "Manage my tasks for today. Check if I can go jogging, and schedule my team meeting."

# Plan and execute the tasks
result = agent(input_text)

# Display the result
print(result)



> Entering new AgentExecutor chain...
I should first read the tasks for today from a file and then check the weather to see if I can go jogging.
Action: read_tasks_from_file
Action Input: today
Observation: [{'title': 'Buy groceries', 'tags': ['shopping']}, {'title': 'Jogging', 'tags': ['outdoor', 'exercise']}, {'title': 'Team meeting', 'tags': ['work']}]
Thought:I need to check the weather to see if I can go jogging and then schedule my team meeting.
Action: check_weather
Action Input: today
Observation: {'queryCost': 1, 'latitude': 51.5064, 'longitude': -0.12721, 'resolvedAddress': 'London, England, United Kingdom', 'address': 'London,UK', 'timezone': 'Europe/London', 'tzoffset': 1.0, 'description': 'Similar temperatures continuing with no rain expected.', 'days': [{'datetime': '2024-09-04', 'datetimeEpoch': 1725404400, 'tempmax': 66.8, 'tempmin': 56.4, 'temp': 61.9, 'feelslikemax': 66.8, 'feelslikemin': 56.4, 'feelslike': 61.9, 'dew': 53.0, 'humidity': 73.4, 'precip': 0.106, 'prec

### Use Built-in Tools for Agents

In [50]:
# calculator Tool
from langchain.agents import initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun

# Initialize the DuckDuckGoSearchRun tool
duckduckgo_tool = DuckDuckGoSearchRun()

# Define a tool for performing a web search
def perform_web_search(query):
    try:
        search_results = duckduckgo_tool.run(query)
        return search_results
    except Exception as e:
        return f"Error: {str(e)}"
    
tools = [
    Tool(
        name="web_search",
        func=perform_web_search,
        description="Performs a web search using DuckDuckGo"
    ),
    Tool(
        name="add_to_calendar",
        func=add_to_calendar,
        description="Adds a task to the calendar"
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Input for the agent
input_text = "Search for the cricket or football match today. if there is a match then displays the match details otherwise schedule a meeting"

# Plan and execute the task
result = agent(input_text)

print(result)



> Entering new AgentExecutor chain...
I need to search for the cricket or football match today first.
Action: web_search
Action Input: "cricket match today"
Observation: Find live score, updates and news for today's cricket matches on ESPNcricinfo. Follow the ICC men's T20 World Cup 2024, The Hundred, One-Day Cup, Shpageeza Cricket League and more. Get Live Cricket Scores, Ball By Ball Commentary, Latest Cricket and Scorecard Updates of all International and domestic cricket matches & related News. ... Today, 06:30 PM . notify. Find out the dates, times and venues of all the cricket matches happening around the world. Check the live scores, results and standings of international, domestic and youth cricket events. Mohammed Shami took a record five-for as India bowled out Sri Lanka for 55, their lowest ODI score, at Wankhede. India qualified for the semi-finals with seven wins in seven matches, while Sri Lanka faced elimination. Cricket Schedule. International Domestic & Others T20 Le